In [29]:
%load_ext autoreload
%autoreload 2

from project.data.data import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preprocess raw data:

In [4]:
processor = RaceDataProcessor()
processor.process_data("RACE", "LON")

## Prepare datasets:

##### All of the following steps should be done from command line. They are here to a quick & dirty heuristic to work with jupyter.

In [30]:
from collections import namedtuple

Hparams = namedtuple("Hparams", ["data_path", "batch_size", "num_workers", "special_tokens", "pretrained_model"])
hparams = Hparams(
    data_path = "LON",
    batch_size = 16,
    num_workers = 6,
    special_tokens = ["[CON]", "[QUE]", "[ANS]", "[DIS]"],
    pretrained_model = "bert-base-cased"
)

In [31]:
def customed_collate_fn(batch, tokenizer):
    """"""
    import torch
    articles = []
    questions = []
    answers = []
    distractors = []

    for item in batch:
        articles.append(" ".join(["<answer>", item["answer"], "<context>", item["article"]]))
        questions.append(item["question"])
    articles = tokenizer(articles, padding=True, 
                                       truncation=True, 
                                       return_tensors="pt", 
                                       pad_to_max_length=True, 
                                       max_length=512)
    questions = tokenizer(questions, padding=True, 
                                       truncation=True, 
                                       return_tensors="pt", 
                                       pad_to_max_length=True, 
                                       max_length=512)
    articles['input_ids'] = torch.squeeze(articles['input_ids'])
    articles['attention_mask'] = torch.squeeze(articles['attention_mask'])
    questions['input_ids'] = torch.squeeze(questions['input_ids'])
    questions['attention_mask'] = torch.squeeze(questions['attention_mask'])
    
    return (articles, questions)

In [32]:
# Create and setup 
data_module = RaceDataModule(hparams, customed_collate_fn)
data_module.prepare_data()
data_module.setup()

In [33]:
# Get data loaders:
trainloader = data_module.train_dataloader()
valloader = data_module.val_dataloader()
testloader = data_module.test_dataloader()

In [34]:
batch = next(iter(trainloader))

In [36]:
type(batch)

list